In [1]:
import pandas as pd
import polars as pl
import numpy as np
import datetime as dt
import asyncio
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mplfinance as mpf
from plotly.subplots import make_subplots
from dash import Dash, dcc, html
from typing import Literal
import pandas_market_calendars as mcal
nse = mcal.get_calendar('NSE')

In [2]:
# Local Variables

# SPREADS
BNF_SPREAD = 100
NIFTY_SPREAD = 50
FNF_SPREAD = 50
MIDCP_SPREAD = 25



In [3]:
async def shortlist_strike_on_premium(
    atm: int,
    spread: int,
    premium_to_find: float,
    index: Literal["bnf", "nifty", "finnifty"],
    datetime: dt.datetime,
    expiry: dt.date,
    option_type: Literal["C", "P"],
    which: Literal["o", "h", "l", "c"] = "o",
    num_strikes: int = 8,
    dir: Literal["up", "down"] | None = None,
) -> int | None:
    strikes_up: list[int] = [atm + (spread * count) for count in range(num_strikes)]
    strikes_down: list[int] = [atm - (spread * count) for count in range(num_strikes)]

    if dir == "up":
        strikes_to_check = list(set(strikes_up))
    elif dir == "down":
        strikes_to_check = list(set(strikes_down))
    else:
        strikes_to_check = list(set(strikes_up + strikes_down))

    strikes_to_check.sort()

    # print(strikes_to_check)
    
    data: list[pl.DataFrame | None] = await asyncio.gather(
        *[
            fetch_option_data(
                index=index,  # type:ignore[arg-type]
                start_date=datetime.date(),
                end_date=datetime.date(),
                start_time=datetime.time(),
                end_time=datetime.time(),
                expiry=expiry,
                strike=strike,
                asset_class=option_type,  # type:ignore[arg-type]
            )
            for strike in strikes_to_check
        ]
    )

    # print(data)

    _schema: SchemaDict | None = None
    for option_data in data:
        # print(len(option_data))
        if len(option_data) == 63:
            # print('Data Not Found for The Options with These Strikes')
            return None
        if option_data is not None:
            _schema = option_data.schema
            break

    if _schema is None:
        return None

    data = [
        d if isinstance(d, pl.DataFrame) else pl.DataFrame(schema=_schema) for d in data
    ]

    # print(data)

    closest_strike = (
        pl.concat(data).with_columns(  # type:ignore[type-var,union-attr]
            (pl.col(which) - premium_to_find).abs().alias("delta"),
        )
    ).sort("delta")[0]["strike"][0]

    return closest_strike

In [4]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = '{:.4f}'.format

In [5]:
import sys
sys.path.append('..')
from tooling.filter import find_atm, option_tool
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm
from tooling.enums import Index, AssetClass, StrikeSpread, Spot

In [6]:
# bnf = pd.read_csv('../data/bnf_min.csv')
bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')

In [7]:
async def get_expiry(f_today):
    
    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    if (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    if (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    if (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24 :
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(start_date=nearest_wednesday, end_date=nearest_wednesday).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

In [8]:
# If Stocks Data ...
bnf_pandas['datetime'] = pd.to_datetime(bnf_pandas['time'])
bnf_pandas['datetime'] = bnf_pandas['datetime'].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas['datetime'].dt.year >= 2023]
bnf_pandas.drop(columns=['time'], inplace=True)
# bnf_pandas

In [9]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# print(bnf.tail())

<class 'polars.dataframe.frame.DataFrame'>


In [10]:
bnf = bnf.with_columns([pl.col('datetime').alias('index')]).drop('datetime')
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf_df = bnf.to_pandas()
print(type(bnf_df))

<class 'pandas.core.frame.DataFrame'>


In [11]:
bnf_df['index'] = pd.to_datetime(bnf_df['index'])
list_of_traded_dates = list(bnf_df['index'].dt.date)
set_of_dates = set(list_of_traded_dates)
# set_of_dates

In [20]:
list_of_dates_to_avoid_bnf = [
    dt.date(2017, 3, 16),
    dt.date(2021, 11, 4),
    dt.date(2022, 10, 24),
]

async def trade(index_df, index_name_str):

    portfolio_value = 10_00_000
    if index_name_str == 'bnf':
        SPREAD = BNF_SPREAD
        index_lev = 6
    else:
        SPREAD = NIFTY_SPREAD
        index_lev = 8

    final_tb = pd.DataFrame()
    trade_book = []

    start_date = dt.date(2017, 1, 1)
    end_date = dt.date(2024, 3, 31)
    
    current_date = start_date

    sl_pct = 25

    max_re_entries = 2

    while current_date <= end_date :

        points_ce = 0
        points_pe = 0
        
        in_trade_ce = False
        in_trade_pe = False
        
        total_ce_trades = 0
        total_pe_trades = 0
        
        ce_re_entry_active = False
        pe_re_entry_active = False
        
        ce_df = None
        pe_df = None
        
        if current_date.weekday() >= 5:
            # print('Skipping Weekend ! ', current_date)
            current_date += dt.timedelta(days=1)
            continue
        elif current_date not in set_of_dates:
            # print('Date Avoided ! ', current_date)
            current_date += dt.timedelta(days=1)
            continue
        elif current_date in list_of_dates_to_avoid_bnf:
            # print('Date Avoided ! ', current_date)
            current_date += dt.timedelta(days=1)
            continue
        else:
            print(current_date)
            # print(index_df.loc[(index_df['datetime'].dt.date == current_date)].iloc[0])
            spot_open = index_df.loc[(index_df['datetime'].dt.date == current_date) & ((index_df['datetime'].dt.time >= dt.time(9, 15)) & (index_df['datetime'].dt.time <= dt.time(15, 30))), 'open'].iloc[0]
            # print(spot_open)
            atm = int(spot_open / 100) * 100
            # print(atm)
            
            current_expiry = await get_expiry(current_date)
            # print(current_expiry)

            days_to_expiry = current_expiry - current_date

            if days_to_expiry.days == 0:
                premium_to_find = 300.0
                re_entry_premium = 250.0
            else:
                premium_to_find = 150
                re_entry_premium = 100
                
            shortlisted_ce = await shortlist_strike_on_premium(
                atm=atm,
                spread=SPREAD,
                premium_to_find=premium_to_find,
                index=index_name_str,
                datetime=dt.datetime.combine(current_date, dt.time(9, 15)),
                expiry=current_expiry,
                option_type='C',
            )
            shortlisted_pe = await shortlist_strike_on_premium(
                atm=atm,
                spread=SPREAD,
                premium_to_find=premium_to_find,
                index=index_name_str,
                datetime=dt.datetime.combine(current_date, dt.time(9, 15)),
                expiry=current_expiry,
                option_type='P',
            )
            # print(shortlisted_ce, shortlisted_pe)

            if (shortlisted_ce is None) or (shortlisted_pe is None):
                # print('Data Not Found , Skipping !' , current_date)
                current_date += dt.timedelta(days=1)
                continue

            ce_df = await fetch_option_data(
                index=index_name_str,
                start_date=current_date,
                end_date=current_date,
                start_time=dt.time(9, 15),
                end_time=dt.time(15, 30),
                expiry=current_expiry,
                strike=shortlisted_ce,
                asset_class='C',
            )
            ce_df = ce_df.to_pandas()
            # print(ce_df.head())
            
            for i in range(0, len(ce_df)):
                # CE Trades
                if total_ce_trades <= max_re_entries:
                    # print(ce_df.iloc[i]['datetime'])
                    # print(ce_df)
                    if not in_trade_ce:
                        if not ce_re_entry_active and (ce_df.iloc[i]['datetime'] == ce_df.loc[(ce_df['datetime'].dt.date == current_date) & ((ce_df['datetime'].dt.time >= dt.time(9, 18)) & (ce_df['datetime'].dt.time <= dt.time(9, 30))) , 'datetime'].iloc[0]):
                            entry_price = ce_df.loc[(ce_df['datetime'].dt.date == current_date) & ((ce_df['datetime'].dt.time >= dt.time(9, 18)) & (ce_df['datetime'].dt.time <= dt.time(9, 30))) , 'o'].iloc[0]
                            entry_time = ce_df.loc[(ce_df['datetime'].dt.date == current_date) & ((ce_df['datetime'].dt.time >= dt.time(9, 18)) & (ce_df['datetime'].dt.time <= dt.time(9, 30))) , 'datetime'].iloc[0]
                            initial_sl = entry_price * (1 + (sl_pct / 100))
                            in_trade_ce = True
                            # print('First Entry Triggered ! ')
    
                        elif ce_re_entry_active:
                            new_spot_open = await fetch_spot_data(
                                instrument=index_name_str,
                                start_date=current_date,
                                start_time=ce_re_entry_time.time(),
                                end_date=current_date,
                                end_time=dt.time(15, 29)
                            )
                            if isinstance(new_spot_open, str):
                                break
                            atm = int(new_spot_open[0]['o'][0] / 100) * 100
                            shortlisted_ce = await shortlist_strike_on_premium(
                                atm=atm,
                                spread=SPREAD,
                                premium_to_find=re_entry_premium,
                                index=index_name_str,
                                datetime=ce_re_entry_time,
                                expiry=current_expiry,
                                option_type='C',
                            )
                            # print(shortlisted_ce)
                            if (shortlisted_ce is None):
                                # print('Data Not Found , Skipping !' , current_date)
                                # current_date += dt.timedelta(days=1)
                                break
                            ce_df2 = await fetch_option_data(
                                index=index_name_str,  # type:ignore[arg-type]
                                start_date=current_date,
                                end_date=current_date,
                                start_time=ce_re_entry_time.time(),
                                end_time=dt.time(15, 30),
                                expiry=current_expiry,
                                strike=shortlisted_ce,
                                asset_class='C',
                            )
                            # print('New CE DF')
                            ce_df = ce_df2.to_pandas()
                            # print(ce_df.head())
                            entry_price = ce_df.iloc[0]['o']
                            entry_time = ce_df.iloc[0]['datetime']
                            initial_sl = entry_price * (1 + (sl_pct * 1.5 / 100))
                            in_trade_ce = True
                            # print('Re-Entry Triggered')
    
                    if in_trade_ce and not ce_re_entry_active:
                        if ce_df.iloc[i]['h'] >= initial_sl:
                            #SL Hit
                            
                            # if ce_re_entry_active:
                            #     # print('CE Re-Entry Deactivated !')
                            #     ce_re_entry_active = False
                                
                            exit_price = initial_sl
                            exit_time = ce_df.iloc[i]['datetime']
                            in_trade_ce = False
                            points_ce = entry_price - exit_price
                            remark = 'SL Hit'
                            ce_re_entry_active = True
                            ce_re_entry_time = ce_df.iloc[i]['datetime'] + dt.timedelta(minutes=1)
    
                        elif ce_df.iloc[i]['datetime'].time() == dt.time(15, 22):
                            # EOD Sq Off
                            exit_price = ce_df.iloc[i]['o']
                            exit_time = ce_df.iloc[i]['datetime']
                            in_trade_ce = False
                            points_ce = entry_price - exit_price
                            remark = 'EOD'
                            total_ce_trades = max_re_entries
                            
                    elif in_trade_ce and ce_re_entry_active:
                        for x in range(0, len(ce_df)):
                            # print(ce_df.iloc[x]['datetime'])
                            if ce_df.iloc[x]['h'] >= initial_sl:
                                #SL Hit
    
                                exit_price = initial_sl
                                exit_time = ce_df.iloc[x]['datetime']
                                in_trade_ce = False
                                points_ce = entry_price - exit_price
                                remark = 'SL Hit'
                                ce_re_entry_active = True
                                ce_re_entry_time = ce_df.iloc[x]['datetime'] + dt.timedelta(minutes=1)
                                
                                if ce_re_entry_active and (total_ce_trades >= max_re_entries):
                                    # print('CE Re-Entry Deactivated !')
                                    ce_re_entry_active = False

                                break

                            elif ce_df.iloc[x]['datetime'].time() == dt.time(15, 22):
                                # EOD Sq Off
                                exit_price = ce_df.iloc[x]['o']
                                exit_time = ce_df.iloc[x]['datetime']
                                in_trade_ce = False
                                points_ce = entry_price - exit_price
                                remark = 'EOD'
                                total_ce_trades = max_re_entries
    
                    if points_ce:
                        contract = await get_option_contract_name(
                            symbol='BANKNIFTY',
                            strike=shortlisted_ce,
                            expiry=current_expiry,
                            opt_type='CE',
                        )
                        qty = portfolio_value * index_lev / spot_open
                        slippage = (entry_price + exit_price) * 0.01
                        final_points = points_ce - slippage
                        trade = {
                            'Contract': contract,
                            'Strike': shortlisted_ce,
                            'Trade Type': 'CE',
                            'Expiry Date': current_expiry,
                            'DTE': current_expiry - entry_time.date(),
                            'Entry Date': entry_time.date(),
                            'Entry Time': entry_time.time(),
                            'Entry Price': entry_price,
                            'Initial SL': initial_sl,
                            'Exit Date': exit_time.date(),
                            'Exit Time': exit_time.time(),
                            'Exit Price': exit_price,
                            'Points Captured': points_ce,
                            'After Costs': final_points,
                            'Remarks': remark,
                            'Qty': qty,
                            'ROI%': (final_points * qty / portfolio_value) * 100,
                            'Trade Year': entry_time.year,
                            'Trade Month': entry_time.month
                        }
                        # print(trade)
                        trade_book.append(trade)
                        points_ce = 0
                        strike = 0
                        in_trade = False
                        remark = ''
                        entry_price = 0
                        initial_sl = 0
                        exit_price = 0
                        entry_time = None
                        exit_time = None
                        total_ce_trades += 1

            pe_df = await fetch_option_data(
                index=index_name_str,  # type:ignore[arg-type]
                start_date=current_date,
                end_date=current_date,
                start_time=dt.time(9, 15),
                end_time=dt.time(15, 30),
                expiry=current_expiry,
                strike=shortlisted_pe,
                asset_class='P',
            )
            pe_df = pe_df.to_pandas()
            # print(pe_df.head())
            
            for i in range(0, len(pe_df)):
                # PE Trades
                if total_pe_trades <= max_re_entries:
                    # print(pe_df.iloc[i]['datetime'])
                    # print(pe_df)
                    if not in_trade_pe:
                        if not pe_re_entry_active and (pe_df.iloc[i]['datetime'] == pe_df.loc[(pe_df['datetime'].dt.date == current_date) & ((pe_df['datetime'].dt.time >= dt.time(9, 18)) & (pe_df['datetime'].dt.time <= dt.time(9, 30))), 'datetime'].iloc[0]):
                            entry_price = pe_df.loc[(pe_df['datetime'].dt.date == current_date) & ((pe_df['datetime'].dt.time >= dt.time(9, 18)) & (pe_df['datetime'].dt.time <= dt.time(9, 30))), 'o'].iloc[0]
                            entry_time = pe_df.loc[(pe_df['datetime'].dt.date == current_date) & ((pe_df['datetime'].dt.time >= dt.time(9, 18)) & (pe_df['datetime'].dt.time <= dt.time(9, 30))), 'datetime'].iloc[0]
                            initial_sl = entry_price * (1 + (sl_pct / 100))
                            in_trade_pe = True
                            # print('First Entry Triggered ! ')
    
                        elif pe_re_entry_active:
                            new_spot_open = await fetch_spot_data(
                                instrument=index_name_str,
                                start_date=current_date,
                                start_time=pe_re_entry_time.time(),
                                end_date=current_date,
                                end_time=dt.time(15, 20)
                            )
                            if isinstance(new_spot_open, str):
                                break
                            atm = int(new_spot_open[0]['o'][0] / 100) * 100
                            shortlisted_pe = await shortlist_strike_on_premium(
                                atm=atm,
                                spread=SPREAD,
                                premium_to_find=re_entry_premium,
                                index=index_name_str,
                                datetime=pe_re_entry_time,
                                expiry=current_expiry,
                                option_type='P',
                            )
                            # print(shortlisted_pe)
                            if (shortlisted_pe is None):
                                # print('Data Not Found , Skipping !' , current_date)
                                # current_date += dt.timedelta(days=1)
                                break
                            pe_df2 = await fetch_option_data(
                                index=index_name_str,  # type:ignore[arg-type]
                                start_date=current_date,
                                end_date=current_date,
                                start_time=pe_re_entry_time.time(),
                                end_time=dt.time(15, 30),
                                expiry=current_expiry,
                                strike=shortlisted_pe,
                                asset_class='P',
                            )
                            # print('New PE DF')
                            pe_df = pe_df2.to_pandas()
                            # print(pe_df.head())
                            entry_price = pe_df.iloc[0]['o']
                            entry_time = pe_df.iloc[0]['datetime']
                            initial_sl = entry_price * (1 + (sl_pct * 1.5 / 100))
                            in_trade_pe = True
                            # print('Re-Entry Triggered')
    
                    if in_trade_pe and not pe_re_entry_active:
                        if pe_df.iloc[i]['h'] >= initial_sl:
                            #SL Hit
                            
                            exit_price = initial_sl
                            exit_time = pe_df.iloc[i]['datetime']
                            in_trade_pe = False
                            points_pe = entry_price - exit_price
                            remark = 'SL Hit'
                            pe_re_entry_active = True
                            pe_re_entry_time = pe_df.iloc[i]['datetime'] + dt.timedelta(minutes=1)
                            
                            # if pe_re_entry_active:
                            #     # print('CE Re-Entry Deactivated !')
                            #     pe_re_entry_active = False    
    
                        elif pe_df.iloc[i]['datetime'].time() == dt.time(15, 22):
                            # EOD Sq Off
                            exit_price = pe_df.iloc[i]['o']
                            exit_time = pe_df.iloc[i]['datetime']
                            in_trade_pe = False
                            points_pe = entry_price - exit_price
                            remark = 'EOD'
                            total_pe_trades = max_re_entries

                    elif in_trade_pe and pe_re_entry_active:
                        for x in range(0, len(pe_df)):
                            if pe_df.iloc[x]['h'] >= initial_sl:
                                #SL Hit
                                
                                exit_price = initial_sl
                                exit_time = pe_df.iloc[x]['datetime']
                                in_trade_pe = False
                                points_pe = entry_price - exit_price
                                remark = 'SL Hit'
                                pe_re_entry_active = True
                                pe_re_entry_time = pe_df.iloc[x]['datetime'] + dt.timedelta(minutes=1)
                                
                                if pe_re_entry_active and total_pe_trades >= max_re_entries:
                                    # print('CE Re-Entry Deactivated !')
                                    pe_re_entry_active = False

                                break
                                    
                            elif pe_df.iloc[x]['datetime'].time() == dt.time(15, 22):
                                # EOD Sq Off
                                exit_price = pe_df.iloc[x]['o']
                                exit_time = pe_df.iloc[x]['datetime']
                                in_trade_pe = False
                                points_pe = entry_price - exit_price
                                remark = 'EOD'
                                total_pe_trades = max_re_entries
    
                    if points_pe:
                        contract = await get_option_contract_name(
                            symbol='BANKNIFTY',
                            strike=shortlisted_pe,
                            expiry=current_expiry,
                            opt_type='PE',
                        )
                        qty = portfolio_value * index_lev / spot_open
                        slippage = (entry_price + exit_price) * 0.01
                        final_points = points_pe - slippage
                        trade = {
                            'Contract': contract,
                            'Strike': shortlisted_pe,
                            'Trade Type': 'PE',
                            'Expiry Date': current_expiry,
                            'DTE': current_expiry - entry_time.date(),
                            'Entry Date': entry_time.date(),
                            'Entry Time': entry_time.time(),
                            'Entry Price': entry_price,
                            'Initial SL': initial_sl,
                            'Exit Date': exit_time.date(),
                            'Exit Time': exit_time.time(),
                            'Exit Price': exit_price,
                            'Points Captured': points_pe,
                            'After Costs': final_points,
                            'Remarks': remark,
                            'Qty': qty,
                            'ROI%': (final_points * qty / portfolio_value) * 100,
                            'Trade Year': entry_time.year,
                            'Trade Month': entry_time.month
                        }
                        # print(trade)
                        trade_book.append(trade)
                        points_pe = 0
                        strike = 0
                        in_trade = False
                        remark = ''
                        entry_price = 0
                        initial_sl = 0
                        exit_price = 0
                        entry_time = None
                        exit_time = None
                        total_pe_trades += 1
                
                tb = pd.DataFrame(trade_book)
            
        final_tb = pd.concat([final_tb, tb])

        tb = pd.DataFrame()
        trade_book = []

        current_date += dt.timedelta(days=1)
            
    return pd.DataFrame(final_tb)

all_trades = await trade(bnf_df, 'bnf')
# all_trades

2023-01-02
2023-01-03
2023-01-04
2023-01-05
2023-01-06
2023-01-09
2023-01-10
2023-01-11
2023-01-12
2023-01-13
2023-01-16
2023-01-17
2023-01-18
2023-01-19
2023-01-20
2023-01-23
2023-01-24
2023-01-25
2023-01-27
2023-01-30
2023-01-31
2023-02-01
2023-02-02
2023-02-03
2023-02-06
2023-02-07
2023-02-08
2023-02-09
2023-02-10
2023-02-13
2023-02-14
2023-02-15
2023-02-16
2023-02-17
2023-02-20
2023-02-21
2023-02-22
2023-02-23
2023-02-24
2023-02-27
2023-02-28
2023-03-01
2023-03-02
2023-03-03
2023-03-06
2023-03-08
2023-03-09
2023-03-10
2023-03-13
2023-03-14
2023-03-15
2023-03-16
2023-03-17
2023-03-20
2023-03-21
2023-03-22
2023-03-23
2023-03-24
2023-03-27
2023-03-28
2023-03-29
2023-03-31
2023-04-03
2023-04-05
2023-04-06
2023-04-10
2023-04-11
2023-04-12
2023-04-13
2023-04-17
2023-04-18
2023-04-19
2023-04-20
2023-04-21
2023-04-24
2023-04-25
2023-04-26
2023-04-27
2023-04-28
2023-05-02
2023-05-03
2023-05-04
2023-05-05
2023-05-08
2023-05-09
2023-05-10
2023-05-11
2023-05-12
2023-05-15
2023-05-16
2023-05-17

In [165]:
# print(bnf_df.head(50))

In [166]:
# print(bnf_df.loc[(bnf_df['datetime'].dt.date == dt.date(2024, 3, 31))].iloc[0])

In [12]:
# all_trades.to_csv('Opt Selling W Re-Entry JJ.csv')

In [168]:
# pnl = all_trades['ROI%'].sum()
# pnl

In [21]:
stats_df8 = pd.DataFrame(index=range(2017, 2025), columns=['Total ROI', 'Total Trades', 'Win Rate', 'Avg Profit% per Trade', 'Avg Loss% per Trade', 'Max Drawdown', 'ROI/DD Ratio'])
combined_df_sorted = all_trades
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted['Trade Year'] == year)]
    
    # Calculate total ROI
    total_roi = year_trades['ROI%'].sum()
    
    # Calculate total number of trades
    total_trades = len(year_trades)
    
    # Calculate win rate
    win_rate = (year_trades['ROI%'] > 0).mean()*100
    
    # Calculate average profit per trade
    avg_profit = year_trades[year_trades['ROI%'] > 0]['ROI%'].mean()
    
    # Calculate average loss per trade
    avg_loss = year_trades[year_trades['ROI%'] < 0]['ROI%'].mean()
    
    # Calculate maximum drawdown
    max_drawdown = (year_trades['ROI%'].cumsum() - year_trades['ROI%'].cumsum().cummax()).min()
    
    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)
    
    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [total_roi, total_trades, win_rate, avg_profit, avg_loss, max_drawdown, roi_dd_ratio]

# Calculate overall statistics
overall_total_roi = stats_df8['Total ROI'].sum()
overall_total_trades = stats_df8['Total Trades'].sum()
overall_win_rate = (combined_df_sorted['ROI%'] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted['ROI%'] > 0]['ROI%'].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted['ROI%'] < 0]['ROI%'].mean()
overall_max_drawdown = (combined_df_sorted['ROI%'].cumsum() - combined_df_sorted['ROI%'].cumsum().cummax()).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc['Overall'] = [overall_total_roi, overall_total_trades, overall_win_rate, overall_avg_profit, overall_avg_loss, overall_max_drawdown, overall_roi_dd_ratio]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,0,NaN,NaN,NaN,NaN,NaN
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN
2022,0.0000,0,NaN,NaN,NaN,NaN,NaN
2023,10.4598,797,39.5232,1.1608,-0.7369,-29.5299,0.3542
2024,5.2944,96,32.2917,1.5695,-0.6671,-9.3555,0.5659
Overall,15.7542,893.0000,38.7458,1.1975,-0.7286,-29.5299,0.5335


In [22]:
stats_of_trades = all_trades
stats_of_trades['Cumulative ROI%'] = stats_of_trades.groupby('DTE')['ROI%'].cumsum()
stats_of_trades['Running Max ROI%'] = stats_of_trades.groupby('DTE')['Cumulative ROI%'].cummax()
stats_of_trades['Drawdown'] = stats_of_trades['Cumulative ROI%'] - stats_of_trades['Running Max ROI%']
max_dd_distribution = stats_of_trades.groupby('DTE')['Drawdown'].min().reset_index()
max_dd_distribution.rename(columns={'Drawdown': 'Max Drawdown'}, inplace=True)
roi_distribution = stats_of_trades.groupby('DTE')['ROI%'].sum().reset_index()
grouped = pd.merge(roi_distribution, max_dd_distribution, on='DTE')
grouped['ROI/DD Ratio'] = grouped['ROI%'] / grouped['Max Drawdown'].abs()
grouped

,DTE,ROI%,Max Drawdown,ROI/DD Ratio
0,0 days,17.9273,-19.0259,0.9423
1,1 days,1.0575,-7.6001,0.1391
2,2 days,4.3229,-4.2158,1.0254
3,3 days,-0.4355,-5.7298,-0.0760
4,5 days,2.9207,-3.0174,0.9679
5,6 days,-10.0387,-11.9765,-0.8382
